# 0. はじめに

### 背景  
・古紙回収においてKSFを実現するには「利用者を増やすこと(離脱率を減らすこと)」「1人あたりの古紙投入量を増やすこと」が必要<br>
・rankシステムは「継続利用」「古紙投入量増加」に寄与している。<br>
・古紙を15kg/月 で3ヵ月連続で投入するとランクアップし、2ヵ月連続で投入しないとランクダウンする。<br>
・1kgあたりの獲得コインが最大6倍になる<br>
  
### 意見  
・「15kg/月」は設定が厳しい可能性。amount_kg_per_yearの最頻値が100kgだったことを考えてもかなりハードル高い。<br>
・古紙投入には限界がある（以降、**古紙投入限界**とよぶ）。古紙を作って捨てようとはならない<br>
  
### 仮説  
・「15kg/月」という数値設定は、排出量の多い一部のユーザにのみしか機能していない可能性がある。<br>
　→15kg/月未満の古紙を投入している層がrankシステムによる継続利用の促進効果を受けていないのではないか<br>
  
### 検証事項  
効果の指標を「継続利用期間（最終更新日-利用開始日）」として、以下のデータ分析を行って仮説を検証。<br>
 
分析1【ランクごとの継続利用期間の比較】<br>
　＜目的＞15kg/月以上出し続ける能力がある人の方が、ない人より継続利用期間が長いことを証明<br>
　＜方法＞・シルバー以上の人と、ブロンズの人の継続利用期間の比較（中央値）<br>


分析2【ぐるっとポンユーザ(15kg/月↑)とRPSユーザ(15kg/月↑)の継続利用期間の比較】<br>
　＜目的＞rankシステムがないと、継続利用期間が短くなることを照明<br>
　＜方法＞・シルバー以上の人と、RPSのみの人の継続利用期間を比較（中央値）<br>
　※アプリ利用者は意識が高い可能性。反実仮想<br>
　※単純比較できるのか怪しい<br>


分析3【ユーザの古紙投入月平均と、ぐるっとポンアクセス月平均の相関】<br>
　＜目的＞古紙投入月平均が15kg/月以上に、正の相関が現れることを証明<br>
　＜方法＞ユーザ毎の古紙投入月平均とぐるっとポンのアクセス月平均を算出し、散布図に描画<br>
 
 
分析4【ユーザの古紙投入月平均と、継続利用期間の相関】<br>
　＜目的＞古紙投入月平均が15kg/月以上に、正の相関が現れることを証明<br>
　＜方法＞ユーザ毎の古紙投入月平均と継続利用期間を算出し、散布図に描画<br>


分析5 :【年代別のランク割合比較】<br>
　＜目的＞子育て世代の古紙投入月平均は低いため、ランク割合も低いことを証明<br>
 　　　　　→ターゲットにrankシステムが刺さっていないことを説明する糧に<br>
　＜方法＞年代別のランク割合を棒グラフで可視化<br>

  
### rankシステムのあるべき姿
・年齢や世帯人数、RPS使用実績から古紙排出限界を予測し、そのユーザーに合ったrank達成目標を設けることで全ユーザーがrankシステムの効果を得る<br>
・↑その上で、ブロンズ、シルバー、ゴールド、プラチナで x kg/月 の傾斜をつけるのもいい？（最初は少なめにするなど）<br>

# 1. 準備

## 1.1. 基本設定

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns #seabornない人はpip installしてね
import os
from datetime import datetime
import numpy as np
from pathlib import Path
from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

# カレントディレクトリを.pyと合わせるために以下を実行
if Path.cwd().name == "notebook":
    os.chdir("..")

# 設定
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 500)
pd.set_option('display.max_columns', 500)

# 浮動小数点数を小数点以下3桁で表示するように設定
pd.set_option('display.float_format', '{:.3f}'.format)

In [ ]:
# Mac Matplotlibのデフォルトフォントをヒラギノ角ゴシックに設定
plt.rcParams['font.family'] = 'Hiragino Sans'

In [ ]:
# Windows MatplotlibのデフォルトフォントをMeiryoに設定
plt.rcParams['font.family'] = 'Meiryo'

## 1.2. csv読み込み

In [ ]:
# point_history.csvの読み込み
df_point_history_sorce = pd.read_csv('data/input/point_history_cleansing.csv')

In [ ]:
# ユーザー基本情報の読み込み
df_user_base_sorce = pd.read_csv("data/input/ユーザー基本情報_2023-12-21.csv", encoding="shift-jis")

## 1.3. データクレンジング

### 1.3.1. df_user_base(ユーザ基本情報)のクレンジング

In [ ]:
# DataFrameのコピーを作成
df_user_base = df_user_base_sorce.copy()

# objectをdatetimeに変更
df_user_base['登録日時'] = pd.to_datetime(df_user_base['登録日時'], errors='coerce')
df_user_base['最終利用日'] = pd.to_datetime(df_user_base['最終利用日'], errors='coerce')
df_user_base['birthday'] = pd.to_datetime(df_user_base['birthday'], errors='coerce')

# 6歳未満(1543個)と100歳以上(12個)を削除
df_user_base = df_user_base[ (df_user_base['birthday'] < pd.to_datetime('2017-01-01')) & (df_user_base['birthday'] > pd.to_datetime('1924-01-01'))]

# df_user_baseに"age"と"age_group"のカラムを追加
df_user_base['age'] = pd.Timestamp.now().year - df_user_base['birthday'].dt.year    # ageの算出・追加

# 年代の算出・追加
bins = [0, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ['0-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']
df_user_base['age_group'] = pd.cut(df_user_base['age'], bins=bins, labels=labels, right=False)

# 今回使用しない可能性が高いカラムは削除
df_user_base = df_user_base.drop(['登録日', 'カード種類', 'スーパー', '都道府県', '市区町村', '登録店舗', 'カード登録日', 'カード更新日', 'birthday','最終利用日'], axis=1)
df_user_base = df_user_base.drop_duplicates(subset='利用者ID', keep='first')
df_user_base = df_user_base.sort_values(by='登録日時')
df_user_base.head(1)

### 1.3.2. df_point_history(point_history.csv)のクレンジング

In [ ]:
# DataFrameのコピーを作成
df_point_history = df_point_history_sorce.copy()

# objectをdatetimeに変更
df_point_history['use_date'] = pd.to_datetime(df_point_history['use_date'], errors='coerce')

# 今回使用しない可能性が高いカラムは削除
df_point_history = df_point_history.drop([
    'Unnamed: 0',
    'id',
    'series_id',
    'shop_id',
    'shop_name',
    'card_id',
    'リサイクル分類ID',
    'amount',
    'created_at',
    'updated_at',
    '支店ID',
    'super',
    'shop_name_1',
    'shop_id_1',
    'updated_at_1',
    'store_opening_time',
    'store_closing_time',
    'created_at_1',
    'rps_opening_time',
    'rps_closing_time',
    'store_latitude',
    'store_longitude',
    'total_amount',], axis=1)
df_point_history = df_point_history.sort_values(by='use_date')

In [ ]:
df_point_history.sort_values(by = 'use_date').tail

### 1.3.3. 分析に必要なカラムの作成

継続利用期間（最終利用日-登録日時）<br>
→最終利用日がバグっているため利用不可<br>
point_historyのuse_date列からRPS最終利用日を抽出して、RPS最終利用日とする　231228 norosen<br>

In [ ]:
# user_idに基づいてグループ化し、各グループの最後の行のみを保持
# 利用回数の算出
count = df_point_history['user_id'].value_counts()
df_point_history['利用回数'] = df_point_history['user_id'].map(count)
df_point_history['利用回数'] = df_point_history['利用回数'].fillna(0)    # NaNを0に変換

In [ ]:
# 総投入量の算出
total_amount = df_point_history.groupby('user_id')['amount_kg'].sum().rename('総投入量')
df_point_history = df_point_history.merge(total_amount, on = 'user_id')

In [ ]:
# RPSの最終利用日を抽出
last_entries = df_point_history.groupby('user_id').last().reset_index()

In [ ]:
# df_user_baseに最終利用日と利用回数と総投入量をマージ
df_user_base = pd.merge(df_user_base, last_entries[['user_id','use_date','利用回数','総投入量']], left_on='利用者ID', right_on='user_id', how='left')
df_user_base = df_user_base.rename(columns={'use_date':'RPS最終利用日'})
df_user_base = df_user_base.drop(columns=['user_id'])

In [ ]:
# 継続利用期間を計算して追加
df_user_base['継続利用期間(月)'] = (df_user_base['RPS最終利用日'] - df_user_base['登録日時']).dt.days / 30  # 月単位で計算
# df_user_base.sort_values(by = '継続利用期間(月)')
# df_user_base[df_user_base['継続利用期間(月)'].isna()]

In [ ]:
df_user_base.head(1)

#### ※df_user_baseとdf_user_basef_point_historyをマージ

In [ ]:
# point_history.csvとユーザー基本情報_2023-12-21.csvをマージ
df_user_base_user_base_merge = pd.merge(df_point_history, df_user_base, left_on='user_id', right_on='利用者ID', how='left')

# 2. 分析

## 分析1
 【ランクごとの継続利用期間（最終更新日-利用開始日）の比較】   
 ＜目的＞15kg/月以上出し続ける能力がある人の方が、ない人より継続利用期間が長いことを証明<br>
　＜方法＞シルバー以上の人と、ブロンズの人の継続利用期間の比較（中央値）<br>

 ＜結果＞ランクが上がるにつれて、中央値は上昇傾向  
 ＜懸念＞データが怪しい。特に継続利用期間の最少が間違っていそう（ゴールドとか）<br>
     　　　　→クレンジングの再見直し

In [ ]:
# 継続利用期間が不正なデータを削除(シルバーは無し)
df_user_base = df_user_base[df_user_base['継続利用期間(月)']>=0]    # 継続利用期間が負の値を削除
df_user_base = df_user_base[~((df_user_base['継続利用期間(月)']<=5) & (df_user_base['現在ランク'] == 'ゴールド'))]    # ゴールドなのに継続利用期間が5ヵ月未満の値を削除
df_user_base = df_user_base[~((df_user_base['継続利用期間(月)']<=6) & (df_user_base['現在ランク'] == 'プラチナ'))]    # プラチナなのに継続利用期間が6ヵ月未満の値を削除

In [ ]:
# ボックスプロットでランク別に継続利用期間を表示
sns.boxplot(x='現在ランク', y='継続利用期間(月)', data=df_user_base, order=['ブロンズ', 'シルバー', 'ゴールド', 'プラチナ'])
plt.xlabel('現在ランク')
plt.ylabel('継続利用期間(月)')
plt.title('ランク別継続利用期間の比較')
plt.show()

In [ ]:
for rank in ['ブロンズ', 'シルバー', 'ゴールド', 'プラチナ']:
    median_duration = df_user_base_user_base_merge_rank_positive[df_user_base_user_base_merge_rank_positive['現在ランク'] == rank]['継続利用期間'].median()
    print(f"{rank}ランクの継続利用期間の中央値: {median_duration}ヶ月")

In [ ]:
# ヒストグラムで分布を表示
plt.figure(figsize=(10, 6))
sns.histplot(df_user_base_user_base_merge_rank_positive, x='継続利用期間', hue='現在ランク', element='step', stat='density', common_norm=False)
plt.xlabel('継続利用期間（月）')
plt.ylabel('密度')
plt.title('シルバーとゴールドランクの継続利用期間の分布')
plt.show()

# KDEプロットで分布を表示
plt.figure(figsize=(10, 6))
sns.kdeplot(data=df_user_base_user_base_merge_rank_positive, x='継続利用期間', hue='現在ランク', common_norm=False)
plt.xlabel('継続利用期間（月）')
plt.ylabel('密度')
plt.title('シルバーとゴールドランクの継続利用期間の分布（KDE）')
plt.show()

## 分析4：年代別のrankの割合

rankはどの時点のものにする？<br>
棒グラフで可視化

## 分析：RFM算出

「最終購入日(Recency)」「購入頻度(Frequency)」「購入金額(Monetary)」の3つでグループ分けするマーケティング分析<br>
→最終利用日、利用回数、総投入量の3つでグループ分けする

In [ ]:
# RPS最終利用日を日付型に変換し、エポックタイム（日数）に変換
df_user_base['RPS最終利用日'] = pd.to_datetime(df_user_base['RPS最終利用日'])
df_user_base['RPS最終利用日(何日前)'] = (pd.Timestamp("2023-12-31") - df_user_base['RPS最終利用日']) // pd.Timedelta('1D')

In [ ]:
df_user_base.sort_values(by='利用回数',ascending = True)

ヒストグラムを表示

In [ ]:
df_user_base_log = df_user_base.copy()
df_user_base_log.loc[df_user_base_log['総投入量'] < 0, '総投入量'] = 1
df_user_base_log.loc[df_user_base_log['RPS最終利用日(何日前)'] < 0, 'RPS最終利用日(何日前)'] = 1



# 利用回数と総投入量の対数変換
df_user_base_log['利用回数'] = np.log10(df_user_base_log['利用回数'])
df_user_base_log['総投入量'] = np.log10(df_user_base_log['総投入量'])
df_user_base_log['RPS最終利用日(何日前)'] = np.log10(df_user_base_log['RPS最終利用日(何日前)'])

# Convert the 'RPS最終利用日' column to datetime
df_user_base_log['RPS最終利用日(何日前)'] = pd.to_datetime(df_user_base_log['RPS最終利用日(何日前)'])

# Plot histograms
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

# Histogram for 'RPS最終利用日'
axs[0].hist(df_user_base_log['RPS最終利用日(何日前)'], bins=10, color='blue', edgecolor='black')
axs[0].set_title('RPS最終利用日のヒストグラム')
axs[0].set_xlabel('日付')
axs[0].set_ylabel('頻度')

# Histogram for '利用回数'
axs[1].hist(df_user_base_log['利用回数'], bins=10, color='green', edgecolor='black')
axs[1].set_title('利用回数のヒストグラム')
axs[1].set_xlabel('利用回数')
axs[1].set_ylabel('頻度')

# Histogram for '総投入量'
axs[2].hist(df_user_base_log['総投入量'], bins=10, color='red', edgecolor='black')
axs[2].set_title('総投入量のヒストグラム')
axs[2].set_xlabel('総投入量')
axs[2].set_ylabel('人数')

plt.tight_layout()
plt.show()

In [ ]:
df_user_base_log = df_user_base.copy()

# 利用回数と総投入量の対数変換
df_user_base_log['利用回数'] = np.log10(df_user_base_log['利用回数'])
df_user_base_log['総投入量'] = np.log10(df_user_base_log['総投入量'])
df_user_base_log['RPS最終利用日(何日前)'] = np.log10(df_user_base_log['RPS最終利用日(何日前)'])


# 3軸グラフの作成
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# プロットデータの設定
x = df_user_base_log['RPS最終利用日(何日前)']
y = df_user_base_log['利用回数']
z = df_user_base_log['総投入量']

# プロット
ax.scatter(x, y, z, c='b', marker='o', alpha = 0.1)

# 軸のラベル設定
ax.set_xlabel('RPS最終利用日（日数）')
ax.set_ylabel('利用回数')
ax.set_zlabel('総投入量')

# タイトル設定
plt.title('RPS最終利用日、利用回数、総投入量の3軸プロット')

# 視点の設定
ax.view_init(30,45)

# 表示
plt.show()

# (参考) visualization_point_history.ipynb の分析

## 2-1. rankごとの平均年間持込量kgの算出

・rank_id毎のrecycle_amount_per_year平均の算出  
・rankが高いほど、年間持ち込み量が多いという仮説を証明する(当然？)  
・rank_idは、0:ブロンズ、1:シルバー、2:ゴールド、3:プラチナ  
・1か月に15kg持ち込み続けるとランクUP。ランクダウンは不明(1か月15kgを達成しないとダウン説、、？)  
※以下の手順を踏むと「rank毎の1回の持ち込み量平均」になってしまうので注意  
　　① amount_kgは1回の持ち込み量。これをrank_id毎にsumする  
　　② rank_idそれぞれの出現回数をsumする  
　　③ ①を②で割る  

In [ ]:
# まずそれぞれのカラムの基本統計量を確認しておく
print('recycle_amount_per_year')
print(df_user_base_user_base_merge['recycle_amount_per_year'].describe())
print('------------------')
print('rank_id')
print(df_user_base_user_base_merge['rank_id'].describe())
print('------------------')
print('rank_idのユニーク')
print(df_user_base_user_base_merge['rank_id'].unique())

※rankは3（プラチナ）が不在な点に注意

In [ ]:
# user_idの重複行を削除（非効率だと思いつつ、、、）
df_user_base_user_base_merge_duplicates_user_id = df_user_base_user_base_merge.drop_duplicates(subset='user_id')

ランク毎の持ち込みamountの平均 = df_user_base_user_base_merge.groupby('rank_id')['recycle_amount_per_year'].mean()
ランク毎の持ち込みamountの平均

### 得られた示唆

・ランクが高いほど、平均年間持ち込み量は多い  
　→「持ち込み量が多いから、ランクが上がった」という解釈の方が正しい？  
 ・rankのカラムのみ、user_infoに紐づけておくと、user_infoの情報量が増えてよいかも


## 2-2. ユーザ個人に着目して指針を得る 

### 2-2-1. トップユーザのamount_kg推移

・point、coin、rankが、ユーザの持ち込み量に影響を与えているのか確認  
・rankを維持したい人


In [ ]:
# 'user_id'の各値の出現回数を計算
user_id_counts = df_user_base_user_base_merge['user_id'].value_counts().reset_index()
user_id_counts.columns = ['user_id', 'count']

# 元のDataFrameにcountをマージ
df_user_base_user_base_merge = df_user_base_user_base_merge.merge(user_id_counts, on='user_id')

# 出現回数に基づいてソート（降順）
df_user_base_user_base_merge = df_user_base_user_base_merge.sort_values(by='count', ascending=False)

In [ ]:
# 重複を削除して、上位100名のcountをtableで俯瞰する
df_user_base_user_base_merged_duplicates_user_id = df_user_base_user_base_merge.drop_duplicates(subset='user_id')
top_100 = df_user_base_user_base_merged_duplicates_user_id.head(100)
top_100

In [ ]:
# top_100の年代別人数を表示
sns.histplot(data=top_100, x='age_group')

plt.title('Age Band Distribution')
plt.xlabel('Age Band')
plt.ylabel('Frequency')

plt.show()

In [ ]:
# 特定の人のamount_kgの推移を確認
# 特定の人のデータのみ抽出
target_user_id = 1152
df_user_base_user_base_merge_target = df_user_base_user_base_merge[df_user_base_user_base_merge['user_id'] == target_user_id]
df_user_base_user_base_merge_target = df_user_base_user_base_merge_target.sort_values(by='use_date', ascending=True)

df_user_base_user_base_merge_target['use_date'] = pd.to_datetime(df_user_base_user_base_merge_target['use_date'])

# use_dateを月ごとに集約（データ数が多すぎて折れ線グラフが見づらい）
df_user_base_user_base_merge_target_monthly= df_user_base_user_base_merge_target[['use_date', 'amount_kg']].resample('M', on='use_date').sum()

display(df_user_base_user_base_merge_target_monthly)

# 集約されたデータで折れ線グラフを作成
plt.figure(figsize=(10, 6))
plt.plot(df_user_base_user_base_merge_target_monthly.index, df_user_base_user_base_merge_target_monthly['amount_kg'], marker='o')
plt.title('Monthly Average of Amount (kg)')
plt.xlabel('Month')
plt.ylabel('Average Amount (kg)')

# データをすべて使って折れ線グラフを作成
plt.figure(figsize=(10, 6))
plt.plot(df_user_base_user_base_merge_target_monthly['use_date'], df_user_base_user_base_merge_target_monthly['rank_id'], marker='o')
plt.title('rank')
plt.xlabel('date')
plt.ylabel('rank')
plt.show()

# coinが付与され始めた時期のデータを表示
filtered_df_user_base_user_base  = df_user_base_user_base_merge_target[df_user_base_user_base_merge_target['coin'] > 0]
filtered_df_user_base_user_base.head()


### まとめ

・15kgを達成しても、すぐにランクが上がらない矛盾あり  
・15kgを未達でもすぐにランクが下がらない矛盾あり  
・アプリを利用していなくても履歴は残る。アプリを利用しているか否かは'coin'(付与コイン)で判断する  

## 2-3. rankシステムの効果測定

・rank_idのレコード

In [ ]:
# TODO: amount_kgの平均を出してしまっているので、user　rank_idごとのamount_kgの合計を出さないといけない？？
# Calculate the average recycling amount for each user and each rank
avg_amount_per_user_rank = df_user_base_user_base_merge.groupby(['user_id', 'rank_id'])['amount_kg'].mean().reset_index()
# avg_amount_per_user_rank = df_user_base_user_base_merge.groupby(['user_id', 'rank_id'])['amount_kg'].sum().reset_index()

# To compare the average recycling amount of different ranks for each user, we need to pivot the data
pivot_table = avg_amount_per_user_rank.pivot(index='user_id', columns='rank_id', values='amount_kg')

# Calculate the difference between each rank's average recycling amount for each user
# For simplicity, we'll calculate the difference between consecutive ranks (rank N and rank N-1)
rank_differences = pivot_table.diff(axis=1)

In [ ]:
rank_differences.head(100)

### 1,2,3ごとの平均値を出してみる

In [ ]:
average_difference_rank = rank_differences.mean(skipna=True)
average_difference_rank

### 1,2,3ごとの合計を出してみる

In [ ]:
# TODO: amount_kgの平均を出してしまっているので、user　rank_idごとのamount_kgの合計を出さないといけない？？
# Calculate the average recycling amount for each user and each rank
sum_amount_per_user_rank = df_user_base_user_base_merge.groupby(['user_id', 'rank_id'])['amount_kg'].sum().reset_index()
# avg_amount_per_user_rank = df_user_base_user_base_merge.groupby(['user_id', 'rank_id'])['amount_kg'].sum().reset_index()

# To compare the average recycling amount of different ranks for each user, we need to pivot the data
pivot_table = sum_amount_per_user_rank.pivot(index='user_id', columns='rank_id', values='amount_kg')

# Calculate the difference between each rank's average recycling amount for each user
# For simplicity, we'll calculate the difference between consecutive ranks (rank N and rank N-1)
rank_differences = pivot_table.diff(axis=1)

In [ ]:
sum_difference_rank = rank_differences.mean(skipna=True)
sum_difference_rank